In [19]:
import json
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob
from elasticsearch import Elasticsearch
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
def load_api():

    consumer_key = '9zLXqMM7Vxlww5235zgzY68bo'
    consumer_secret = 'Z5eQzWrhJhfvPDVqIIwHYMvXT1WN9kCMkmilXwuoEnTqDOQPH5'
    access_token = '959344336549838848-f09D838e9YXiU1o4ivFlaTf2Is9KiY5'
    access_token_secret = 'lr9i6Xr8CKPSnp7RPPMmd3lYJc4ETK1qauS568KxTuNhI'
    auth=OAuthHandler(consumer_key,consumer_secret)
    auth.set_access_token(access_token,access_token_secret)
    return tweepy.API(auth)

In [21]:
es = Elasticsearch()

In [22]:
class TweetStreamListener(StreamListener):

    # on success
    def on_data(self, data):

        # decode json
        dict_data = json.loads(data)

        # pass tweet into TextBlob
        tweet = TextBlob(dict_data["text"])

        # output sentiment polarity
        print(tweet.sentiment.polarity)

        # determine if sentiment is positive, negative, or neutral
        if tweet.sentiment.polarity < 0:
            sentiment = "negative"
        elif tweet.sentiment.polarity == 0:
            sentiment = "neutral"
        else:
            sentiment = "positive"

        # output sentiment
        print(sentiment)
        es.index(index="sentiment",
                 doc_type="test-type",
                 body={"author": dict_data["user"]["screen_name"],
                       "date": dict_data["created_at"],
                       "message": dict_data["text"],
                       "polarity": tweet.sentiment.polarity,
                       "subjectivity": tweet.sentiment.subjectivity,
                       "sentiment": sentiment})
        return True
     # on failure
    def on_error(self, status):
        print(status)



In [24]:
if __name__ == '__main__':

    # create instance of the tweepy tweet stream listener
    listener = TweetStreamListener()

    # create instance of the tweepy stream
    stream = Stream(auth, listener)

    # search twitter for "congress" keyword
    stream.filter(track=['Buhari'])

0.0
neutral


POST http://localhost:9200/sentiment/test-type [status:N/A request:2.032s]
Traceback (most recent call last):
  File "c:\users\abayomi\appdata\local\programs\python\python35\lib\site-packages\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "c:\users\abayomi\appdata\local\programs\python\python35\lib\site-packages\urllib3\util\connection.py", line 83, in create_connection
    raise err
  File "c:\users\abayomi\appdata\local\programs\python\python35\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\abayomi\appdata\local\programs\python\python35\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 166, in perform_request
    response = 

POST http://localhost:9200/sentiment/test-type [status:N/A request:2.008s]
Traceback (most recent call last):
  File "c:\users\abayomi\appdata\local\programs\python\python35\lib\site-packages\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "c:\users\abayomi\appdata\local\programs\python\python35\lib\site-packages\urllib3\util\connection.py", line 83, in create_connection
    raise err
  File "c:\users\abayomi\appdata\local\programs\python\python35\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\abayomi\appdata\local\programs\python\python35\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 166, in perform_request
    response = 

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x0000007359393358>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x0000007359393358>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it)